# 引入工具包

In [1]:
import numpy as np  # linear algebra
import pandas as pd  #
from datetime import datetime, date, timedelta
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,Ridge,Lasso,ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor,RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.svm import SVR, LinearSVC
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
import time
from itertools import product
import datetime as dt
import calendar
import gc

RANDOM_SEED = 42
# 引入中文字体
from matplotlib.font_manager import FontProperties
myfont = FontProperties(fname="/home/aistudio/NotoSansCJKsc-Light.otf", size=12)

# 加载、分析数据集
## 加载数据集

In [ ]:
PATH = './data/data19383/'
train = pd.read_csv(PATH + 'train.csv')
# train = pd.read_csv('./data/data19383/train.csv', usecols=[2, 3, 4, 6, 7, 18])
# set index to ID to avoid droping it later
# 把测试集的id列作为索引，防止误删
test  = pd.read_csv(PATH + 'submission.csv').set_index('customer_id')

In [ ]:
# 对于特别大的文件，我们需要做一些内存检查
mem_train = train.memory_usage(index=True).sum()
mem_test=test.memory_usage(index=True).sum()
print(u"训练集使用内容 "+ str(mem_train/ 1024**2)+" MB")
print(u"测试集使用内存 "+ str(mem_test/ 1024**2)+" MB")

### 内存优化脚本
- 参考[缓解pandas中DataFrame占用内存过大的问题](https://blog.csdn.net/wj1066/article/details/81124959)
- 效果非常显著，有效避免内存溢出

In [ ]:
# @from: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65/code
# @liscense: Apache 2.0
# @author: weijian
def reduce_mem_usage(props):
    # 计算当前内存
    start_mem_usg = props.memory_usage().sum() / 1024 ** 2
    print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
    # 哪些列包含空值，空值用-999填充。why：因为np.nan当做float处理
    NAlist = []
    for col in props.columns:
        # 这里只过滤了objectd格式，如果你的代码中还包含其他类型，请一并过滤
        if (props[col].dtypes != object):
            
            # print("**************************")
            # print("columns: ", col)
            # print("dtype before", props[col].dtype)
            
            # 判断是否是int类型
            isInt = False
            mmax = props[col].max()
            mmin = props[col].min()
            
            # Integer does not support NA, therefore Na needs to be filled
            if not np.isfinite(props[col]).all():
                NAlist.append(col)
                props[col].fillna(-999, inplace=True) # 用-999填充
                
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = np.fabs(props[col] - asint)
            result = result.sum()
            if result < 0.01: # 绝对误差和小于0.01认为可以转换的，要根据task修改
                isInt = True
            
            # make interger / unsigned Integer datatypes
            if isInt:
                if mmin >= 0: # 最小值大于0，转换成无符号整型
                    if mmax <= 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mmax <= 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mmax <= 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else: # 转换成有符号整型
                    if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)  
            else: # 注意：这里对于float都转换成float16，需要根据你的情况自己更改
                props[col] = props[col].astype(np.float16)
            
            # print("dtype after", props[col].dtype)
            # print("********************************")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
# 处理id字段
train['order_detail_id'] = train['order_detail_id'].astype(np.uint32)
train['order_id'] = train['order_id'].astype(np.uint32)
train['customer_id'] = train['customer_id'].astype(np.uint32)
train['goods_id'] = train['goods_id'].astype(np.uint32)
train['goods_class_id'] = train['goods_class_id'].astype(np.uint32)
train['member_id'] = train['member_id'].astype(np.uint32)
# 处理状态字段，这里同时处理空值，将空值置为0
train['order_status'] = train['order_status'].astype(np.uint8)
train['goods_has_discount'] = train['goods_has_discount'].astype(np.uint8)
train["is_member_actived"].fillna(0, inplace=True)
train["is_member_actived"]=train["is_member_actived"].astype(np.int8)
train["member_status"].fillna(0, inplace=True)
train["member_status"]=train["member_status"].astype(np.int8)
train["customer_gender"].fillna(0, inplace=True)
train["customer_gender"]=train["customer_gender"].astype(np.int8)
train['is_customer_rate'] = train['is_customer_rate'].astype(np.uint8)
train['order_detail_status'] = train['order_detail_status'].astype(np.uint8)
# 处理日期
train['goods_list_time']=pd.to_datetime(train['goods_list_time'],format="%Y-%m-%d")
train['order_pay_time']=pd.to_datetime(train['order_pay_time'],format="%Y-%m-%d")
train['goods_delist_time']=pd.to_datetime(train['goods_delist_time'],format="%Y-%m-%d")
# 检查内存使用
mem_train = train.memory_usage(index=True).sum()
mem_test=test.memory_usage(index=True).sum()
print(u"训练集使用内容 "+ str(mem_train/ 1024**2)+" MB")
print(u"测试集使用内存 "+ str(mem_test/ 1024**2)+" MB")

In [ ]:
train['customer_city_id'] = LabelEncoder().fit_transform(train['customer_city'].astype(str))
train['customer_province_id'] = LabelEncoder().fit_transform(train['customer_province'].astype(str))

以下内容参考并改造自[Kaggle竞赛宝典](https://github.com/dayeren/Kaggle_Competition_Treasure/tree/master/Recommendations/Instacart)
# 特征工程(基础特征更新中)

目前很多比赛都还是特征为王的比赛，针对许多传统的比赛，因为存在很多封装好的工具包,例如XGBoost,LightGBM等工具包,所以很多做数据分析建模或者数据竞赛的人的任务往往就会相对简单，这个时候因为模型变化不大,那么我们的任务往往就会演变为两大模块,特征工程+集成，而特征工程又很难找到非常好的书籍进行系统的学习,所以很多时候网上也都将特征工程称之为艺术。

闲话不多说,下面我们还是以推荐问题为例,介绍基本的推荐算法的特征工程。

我们将该赛题的特征工程粗分为下面几个模块。

1. 用户特征;
2. 产品特征;
3. 用户产品特征;

## 用户特征 

In [ ]:
train.columns

当前数据集中没有days_since_prior_order这个字段，需要手动构造。

In [ ]:
# orders = train[train.order_status<101][train.order_pay_time>'2013-01-01'][['customer_id','order_id','order_pay_time','order_total_num',
# 'order_amount','order_total_payment','order_total_discount','order_count']]
orders = train[train.order_pay_time>'2013-01-01'][['customer_id','order_id','order_pay_time','order_total_num','order_amount','order_total_payment','order_total_discount','order_count']]

In [ ]:
# 删除重复行
orders = orders.drop_duplicates(['order_id'])

In [ ]:
orders  = orders.set_index(['customer_id','order_id'])
# 同一用户按下单时间排序
orders = orders.sort_values(['customer_id','order_pay_time'])
# 把customer_id和order_pay_time两列数据都下移一行，order_pay_time用来算时间差，customer_id用于判断
orders['last_pay_time'] = orders['order_pay_time'].shift()
# orders = tmp[['order_pay_time','last_pay_time']]
orders.reset_index(inplace=True)
orders['customer_id_before'] = orders['customer_id'].shift().fillna(0).astype(np.uint32)

In [ ]:
# orders['days_since_prior_order'] = (pd.to_datetime(orders['order_pay_time'])-pd.to_datetime(orders['last_pay_time'])).astype('timedelta64[D]')

In [ ]:
# 定义一个赋值函数，同时起到判断是否填入空值和给days_since_prior_order字段赋值的作用
def fill_row(x, y, a, b):
    # print(t)
    if x != y:
        t = np.nan
    else:
        t = (a - b).days
    return t

orders['days_since_prior_order'] = orders.apply(lambda row: fill_row(row['customer_id'],row['customer_id_before'],row['order_pay_time'],row['last_pay_time']), axis=1)

In [ ]:
# 删除中间过渡列
del orders['customer_id_before']
del orders['last_pay_time']
gc.collect()

至此，orders表的格式和模板一致了，后续的特征工程也会比较容易

### 用户购物的总次数

In [ ]:
user_fea = pd.DataFrame()
user_fea['customer_id'] = orders['customer_id'].unique()
user_fea = user_fea[['customer_id']].sort_values('customer_id')

In [ ]:
user_fea['user_orderid_count'] = orders.groupby('customer_id')['order_id'].count().values 

### 用户购物频繁度

1. 如果用户平均相隔天数少,往往说明用户是爱频繁购物的，那么他很有可能会不断地去消费某些产品;
2. 如果用户距离上次的购物最大天数都很小,那么往往说明客户是非常爱购物的,和均值特征是类似的一种.
3. 如果用户距离上次购物的时间的方差较小,说明用户购物有较强的周期性.
4. 同样的,用户距离上一次购买产品的众数也是此类特征之一.(该数据集中出现报错，暂时未实现)

In [ ]:
user_fea['user_days_since_prior_order_mean'] = orders.groupby('customer_id')['days_since_prior_order'].mean().values
user_fea['user_days_since_prior_order_max'] = orders.groupby('customer_id')['days_since_prior_order'].max().values
user_fea['user_days_since_prior_order_std'] = orders.groupby('customer_id')['days_since_prior_order'].std().values

计算众数报错的原因可能是因为该数据集中只购买一次的用户很多，众数不能用原来的办法算。

In [ ]:
# user_fea['user_days_since_prior_order_mode'] = orders.groupby('customer_id')['days_since_prior_order'].apply(lambda x: x.mode()[0]).values

### 用户时间喜好
- 不同用户购物的时间往往不同,很多用户喜欢深夜购物,有的用户则往往喜欢周五购物,这些不同的时间点往往也是一个非常重要的选择,比如有些用户喜欢周五买香蕉什么的生活习惯,可以对预测带来非常大的帮助。有人则喜欢深夜购物等等。

In [ ]:
orders['order_hour_of_day'] = pd.DatetimeIndex(orders['order_pay_time']).hour
orders['order_dow'] = pd.DatetimeIndex(orders['order_pay_time']).dayofweek

In [ ]:
user_fea['user_order_dow_mode'] = orders.groupby('customer_id')['order_dow'].apply(lambda x: x.mode()[0]).values
user_fea['user_order_hour_of_day_mode'] = orders.groupby('customer_id')['order_hour_of_day'].apply(lambda x: x.mode()[0]).values

- 补充上一个二阶的用户时间喜好特征,用来表示用户喜欢在每周晚上几点开始购物.

In [ ]:
orders['dow_hour'] = orders['order_dow'].values * 25 + orders['order_hour_of_day'].values
user_fea['user_dow_hour_mode'] = orders.groupby('customer_id')['dow_hour'].apply(lambda x: x.mode()[0]).values

### 用户商品喜好特征
- 注意此处是从train中提取特征,这样数据量更大,因为test中的用户和train中一致,所以不会造成偏差。

此处我们用如下的几类特征来表示用户商品的特征.
1. 用户购买不同产品的数量
2. 用户购买最多的三种产品,如果用户只购买了两类产品,那么最后一种产品我们用-1来表示.
3. 用户购买所有产品的数量
4. 用户平均每一单购买的产品数量

In [ ]:
order_products__prior = train[train.order_pay_time>'2013-01-01']

In [ ]:
user_fea['user_product_nunique'] = order_products__prior.groupby('customer_id')['goods_id'].nunique().sort_index().values

In [ ]:
def mode_N(x,i):
    m = x.value_counts().index
    if len(m) > i: 
        return m[i] 
    return -1
user_fea['user_product_mode'] = order_products__prior.groupby('customer_id')['goods_id'].apply(lambda x: x.mode()[0]).sort_index().values
user_fea['user_product_mode_1'] = order_products__prior.groupby('customer_id')['goods_id'].apply(lambda x: mode_N(x,1)).sort_index().values 
user_fea['user_product_mode_2'] = order_products__prior.groupby('customer_id')['goods_id'].apply(lambda x: mode_N(x,2)).sort_index().values

In [ ]:
user_fea['user_product_count'] = order_products__prior.groupby('customer_id')['goods_id'].count().sort_index().values

In [ ]:
user_fea['user_product_orderid_ratio'] = user_fea['user_product_count'] / user_fea['user_orderid_count']

### 用户购物疯狂性特征
1. 用户订单中的商品最多数
2. 用户每次平均购买的商品量
3. 用户购买商品的1/4,3/4位数

In [ ]:
tmp = order_products__prior.groupby(['customer_id','order_id'])['order_total_num'].max().reset_index()

In [ ]:
user_fea['user_add_to_cart_order_max'] = tmp.groupby('customer_id')['order_total_num'].max().sort_index().values
user_fea['user_add_to_cart_order_mean'] = tmp.groupby('customer_id')['order_total_num'].mean().sort_index().values
user_fea['user_add_to_cart_order_quantile_25'] = tmp.groupby('customer_id')['order_total_num'].quantile().sort_index().values
user_fea['user_add_to_cart_order_quantile_75'] = tmp.groupby('customer_id')['order_total_num'].quantile(0.75).sort_index().values

In [ ]:
del tmp
gc.collect()

### 用户购买商品的重购率

我们用用户购买的所有商品中的重新购买的和以及均值来表示

In [ ]:
tmp = order_products__prior[['order_id','customer_id','goods_id']]
tmp['reordered'] = 1

In [ ]:
# 新增reordered字段
tmp = tmp.groupby(['order_id','customer_id','goods_id']).agg({'reordered':'sum'})
tmp.columns = ['reordered']
tmp.reset_index(inplace=True)
tmp['reordered'] = tmp.apply(lambda row: row['reordered']-1, axis=1)

In [ ]:
tmp = tmp.sort_values('customer_id')

In [ ]:
user_fea['user_reordered_sum'] = tmp.groupby('customer_id')['reordered'].sum().sort_index().values
user_fea['user_reordered_mean'] = tmp.groupby('customer_id')['reordered'].mean().sort_index().values

In [ ]:
del tmp
gc.collect()

In [ ]:
user_fea.to_pickle('user_fea.pkl')

完成这部分内容后，整合单时间滑窗（付款金额）的验证集`auc`达到0.97，但是测试集效果依然很差；用了双时间滑窗后的表现有了大幅提升，`weighted_ce`一度达到2.27432，已经非常接近没有整合用户画像时的表现，同时，用户基本特征的权重仍然非常靠前，猜测是发生了过拟合。

这里产生了一个用户画像的用法的思路，可能考虑做一个聚类，对用户进行分类会比较合适？

## 基于产品的特征

上面我们对用户特征进行了简单的提取,分别针对用户的购买时间偏好,用户对商品的喜好,重构的情况等等出发进行特征的构建。

下面我们从商品的特征出发进行商品特征的构建。虽然下面的特征都可以认为是产品的热度&受欢迎度特征等,但我们还是尽可能将其分开解释。

### 产品的热度
1. 产品被买多少次
2. 产品被多少不同用户所购买
3. 产品被添加到篮子中的顺序的均值 & 标准差

In [ ]:
product_fea = pd.DataFrame()
product_fea['product_id'] = order_products__prior['goods_id'].unique() 
product_fea= product_fea.sort_values('product_id')

In [ ]:
product_fea['product_count'] = order_products__prior.groupby('goods_id')['customer_id'].count().sort_index().values

In [ ]:
product_fea['product_order_nunqiue'] = order_products__prior.groupby('goods_id')['order_id'].nunique().sort_index().values
product_fea['product_user_nunqiue']  = order_products__prior.groupby('goods_id')['customer_id'].nunique().sort_index().values

In [ ]:
product_fea['product_add_to_cart_order_mean'] = order_products__prior.groupby('goods_id')['order_detail_goods_num'].mean().sort_index().values 
product_fea['product_add_to_cart_order_std']  = order_products__prior.groupby('goods_id')['order_detail_goods_num'].std().sort_index().values 

### 产品受欢迎的时间段
1. 产品被购买最多的两天
2. 产品被购买最多的两个小时段
3. 产品被购买距离上次购买最多的两个时间段
4. 产品距离上次被购买的均值以及均方差

In [ ]:
order_products__prior['order_hour_of_day'] = pd.DatetimeIndex(order_products__prior['order_pay_time']).hour
order_products__prior['order_dow'] = pd.DatetimeIndex(order_products__prior['order_pay_time']).dayofweek
order_products__prior['dow_hour'] = order_products__prior['order_dow'].values * 25 + order_products__prior['order_hour_of_day'].values

In [ ]:
product_fea['product_dow_mode'] = order_products__prior.groupby('goods_id')['order_dow'].apply(lambda x: x.mode()[0]).sort_index().values
product_fea['product_dow_mode_1'] = order_products__prior.groupby('goods_id')['order_dow'].apply(lambda x: mode_N(x,1)).sort_index().values

In [ ]:
product_fea['product_hour_of_day_mode'] = order_products__prior.groupby('goods_id')['order_hour_of_day'].apply(lambda x: x.mode()[0]).sort_index().values
product_fea['product_hour_of_day_mode_1'] = order_products__prior.groupby('goods_id')['order_hour_of_day'].apply(lambda x: mode_N(x,1)).sort_index().values

In [ ]:
tmp = orders[['order_id','days_since_prior_order']]
order_products__prior = pd.merge(order_products__prior,tmp,how='left')
del tmp
gc.collect()

In [ ]:
product_fea['product_days_since_prior_order_mode'] = order_products__prior.groupby('goods_id')['days_since_prior_order'].apply(lambda x: mode_N(x,0)).sort_index().values
product_fea['product_days_since_prior_order_mode_1'] = order_products__prior.groupby('goods_id')['days_since_prior_order'].apply(lambda x: mode_N(x,1)).sort_index().values

In [ ]:
product_fea['product_dow_hour_mode'] = order_products__prior.groupby('goods_id')['dow_hour'].apply(lambda x: mode_N(x,0)).sort_index().values
product_fea['product_dow_hour_mode_1'] = order_products__prior.groupby('goods_id')['dow_hour'].apply(lambda x: mode_N(x,1)).sort_index().values

In [ ]:
product_fea['product_days_since_prior_order_mean'] = order_products__prior.groupby('goods_id')['days_since_prior_order'].mean().sort_index().values 
product_fea['product_days_since_prior_order_std'] = order_products__prior.groupby('goods_id')['days_since_prior_order'].std().sort_index().values 

### 产品的被重购率

产品的重购率,标准差以及和。

In [ ]:
tmp = order_products__prior[['order_id','customer_id','goods_id']]
tmp['reordered'] = 1
# 新增reordered字段
tmp = tmp.groupby(['order_id','customer_id','goods_id']).agg({'reordered':'sum'})
tmp.columns = ['reordered']
tmp.reset_index(inplace=True)
tmp['reordered'] = tmp.apply(lambda row: row['reordered']-1, axis=1)
tmp = tmp.sort_values('customer_id')

In [ ]:
product_fea['product_reordered_mean'] = tmp.groupby('goods_id')['reordered'].mean().sort_index().values 
product_fea['product_reordered_std'] = tmp.groupby('goods_id')['reordered'].std().sort_index().values 
product_fea['product_reordered_sum'] = tmp.groupby('goods_id')['reordered'].sum().sort_index().values 

In [ ]:
del tmp
gc.collect()

In [ ]:
product_fea.to_pickle('product_fea.pkl')